In [10]:
import requests

In [11]:
standings_url_2324 = "https://fbref.com/en/comps/20/Bundesliga-Stats"

In [12]:
#standings_url_2223 = "https://fbref.com/en/comps/20/2022-2023/2022-2023-Bundesliga-Stats"

In [13]:
data = requests.get(standings_url_2324)

In [14]:
from bs4 import BeautifulSoup

In [1]:
soup = BeautifulSoup(data.text)
#select the html for every club
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [7]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [8]:
data = requests.get(team_urls[0])

In [9]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [10]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [11]:
data = requests.get(f"https://fbref.com{links[0]}")

In [12]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [13]:
#get rid of multilevel index
shooting.columns = shooting.columns.droplevel()

In [14]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2023-08-12,13:00,DFB-Pokal,Round of 64,Sat,Away,W,4,0,TSG Balingen,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2023-08-19,15:30,Bundesliga,Matchweek 1,Sat,Home,W,5,0,Bochum,...,16.6,0.0,0,0,3.0,3.0,0.15,2.0,2.0,Match Report
2,NaN,NaN,NaN,NaN,NaN,NaN,2-0-0,9,0,NaN,...,16.6,0.0,0,0,3.0,3.0,0.15,6.0,6.0,NaN


In [15]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [16]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-12,13:00,DFB-Pokal,Round of 64,Sat,Away,W,4.0,0.0,TSG Balingen,...,4-3-3,Robert Kampka,Match Report,NaN,24,10,NaN,NaN,0,0
1,2023-08-19,15:30,Bundesliga,Matchweek 1,Sat,Home,W,5.0,0.0,Bochum,...,4-2-3-1,Daniel Siebert,Match Report,NaN,21,9,16.6,0.0,0,0


Scaling up

In [20]:
years = list(range(2024, 2021, -1))
all_matches = []

In [21]:
standings_url = "https://fbref.com/en/comps/20/Bundesliga-Stats"

In [2]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Bundesliga"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [19]:
len(all_matches)

36

In [20]:
match_df = pd.concat(all_matches)

In [21]:
match_df.columns = [c.lower() for c in match_df.columns]

In [22]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2022-08-05,20:30,Bundesliga,Matchweek 1,Fri,Away,W,6.0,1.0,Eint Frankfurt,...,Match Report,NaN,23.0,10.0,16.6,1.0,0.0,0.0,2023,Bayern Munich
2,2022-08-14,17:30,Bundesliga,Matchweek 2,Sun,Home,W,2.0,0.0,Wolfsburg,...,Match Report,NaN,24.0,8.0,18.4,0.0,0.0,0.0,2023,Bayern Munich
3,2022-08-21,17:30,Bundesliga,Matchweek 3,Sun,Away,W,7.0,0.0,Bochum,...,Match Report,NaN,21.0,11.0,18.3,1.0,1.0,1.0,2023,Bayern Munich
4,2022-08-27,18:30,Bundesliga,Matchweek 4,Sat,Home,D,1.0,1.0,M'Gladbach,...,Match Report,NaN,33.0,20.0,17.4,1.0,0.0,0.0,2023,Bayern Munich
6,2022-09-03,15:30,Bundesliga,Matchweek 5,Sat,Away,D,1.0,1.0,Union Berlin,...,Match Report,NaN,21.0,6.0,18.1,0.0,0.0,0.0,2023,Bayern Munich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2022-04-17,17:30,Bundesliga,Matchweek 30,Sun,Away,D,0,0,Hoffenheim,...,Match Report,NaN,8.0,1.0,24.2,1.0,0.0,0.0,2022,Greuther Furth
31,2022-04-23,15:30,Bundesliga,Matchweek 31,Sat,Home,L,1,4,Leverkusen,...,Match Report,NaN,8.0,3.0,15.6,0.0,0.0,0.0,2022,Greuther Furth
32,2022-04-29,20:30,Bundesliga,Matchweek 32,Fri,Away,D,1,1,Union Berlin,...,Match Report,NaN,7.0,2.0,22.4,0.0,0.0,0.0,2022,Greuther Furth
33,2022-05-07,15:30,Bundesliga,Matchweek 33,Sat,Home,L,1,3,Dortmund,...,Match Report,NaN,8.0,3.0,20.0,0.0,0.0,0.0,2022,Greuther Furth


In [23]:
match_df.to_csv("matches.csv")